<a href="https://colab.research.google.com/github/ameena007/learning-terraform-3087701/blob/main/guardrails_with_T5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[SentencePiece]

!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensor

https://towardsdatascience.com/poor-mans-gpt-3-few-shot-text-generation-with-t5-transformer-51f1b01f843e

https://colab.research.google.com/drive/1r4NYdk47vsayOELyIGMeoR9xGf7PYveE?usp=sharing#scrollTo=tpUH01FvMuus


In [3]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
guardrails_data_tuples = [
    ("What is Account discovery in Aria Guardrails", "VMware Aria Guardrails discovers accounts and displays them in the cloud accounts, with their associated project, number of drifts that occurred, number of desired states. Each account displays a graph of the guardrail drift correct over time, the state name and comments, and the ability to display the states with the associated policy, resource name, and role name.For an example of account discovery and drifts identified"),
    ("What is Bootstrap in Aria Guardrails","With VMware Aria Guardrails bootstrap templates, you can create an Amazon AWS account or a Microsoft Azure account and secure it with governance of your network, security, cost, performance, and config. The bootstrap templates create the account and apply governance to it. For example, for AWS, the bootstrap templates create the AWS organizational units (OU), AWS member account, and AWS landing zone, and apply policies to the OU. You can then create desired states and run them."),
    ("What is Cloud account in Aria Guardrails","An area of the VMware Aria Guardrails user interface that displays all the discovered accounts, the projects that they are associated with, the number of drifts detected, and the number of states enforced. As an administrator, you can assign the cloud account to another project. You can also display details about the detected drifts in the account, along with desired states and projects."),
    ("What is Desired state in Aria Guardrails","The combination of a VMware Aria Guardrails template with the required input parameters and credentials for creating the infrastructure and enforcing policies on it. Desired states can run on-demand or in response to an event. You can view the desired states for each of your accounts in the cloud accounts. You can identify who ran the desired state, start or stop enforcing the desired state, and view the enforcement history with all the details about the desired state."),
    ("What is Drift in Aria Guardrails", "Drifts are detected deviations from the established guardrails policies. To keep accounts compliant, and ensure that drifts do not occur, line of business owners enforce guardrails on existing deployments on a recurring schedule. VMware Aria Guardrails monitors and detects the drifts of your policies and displays them in the cloud accounts. To view the drifts in your account, you must have the correct permissions."),
    ("What is Enforcement in Aria Guardrails", "A point in time that a VMware Aria Guardrails desired state ran. The desired state produces Idem state events that users can subscribe to for various use cases."),
    ("What is Findings in Aria Guardrails", "Violations that VMware Aria Guardrails discovered. Findings are driven by VMware Aria Automation for Secure Clouds and appear as drifts from the desired state in the findings for the cloud accounts. When you view the findings for an account, the results indicate the number of applied states over time. The associated desired states display the history and the results by state. The results of the desired state that ran include overall results and results by state."),
    ("What is Aria Guardrails", "A high-level rule or policy that provides ongoing governance for your public cloud accounts. Guardrails enable you to express the desired state of your account based on policies. You can enforce guardrails at multiple resource group levels in the public cloud. VMware Aria Guardrails provides multiple types of enforcement: Preventive guardrails: Ensure compliance of the target account by preventing actions that can lead to policy violations. Extra policy engines are not required for enforcement. For example, for provisioning resources in a landing zone, you set up policies for IAM, network, logging, monitoring, and so on, by using the minimum set of privileges that keep users from violating policies. Minimum privileges can include setting up an AWS IAM role that only allows provisioning EC2 instances of type *.large. Detective guardrails: Ensure compliance by detecting the non-compliance of resources, such as policy violations, generating alerts, and running remediation actions that fix violations. Users might implement a policy engine like Open Policy Agent (OPA), VMware Aria Automation for Secure Clouds, VMware Aria Automation for Secure Hosts, or any of several others for detecting, and in some cases remediating the security, cost, performance, and other categories of policy violations. For example, for continuous policy enforcement, detect policy violations in the account. Also, set up actions that can run in an automated manner in response to the violation alerts, such as terminating development VMs that become idle for longer than 30 days."),
    ("What is Guardrails templates","A collection of Idem SLS files including a landing zone, preventive guardrails, detective guardrails, and actions that can run that remediate a guardrails violation, all expressed as code by Idem states.")
]

In [7]:
t5_model.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in guardrails_data_tuples:
    input_sent = input+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


epoch  0


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
test_sent = 'What is template. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

A collection of Idem SLS files that users can subscribe to for various use cases. Templates can run on-demand or in response to an event.
A collection of Idem SLS files that users can subscribe to for free. Templates can run on-demand or in response to an event.
A high-level rule or policy that provides ongoing governance for your network. Templates can run on-demand or in response to an event.


In [ ]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
from huggingface_hub import HfApi

tokenizer.save_pretrained("vishal_model")
t5_model.save_pretrained("vishal_model", max_shard_size="2000MB")

api = HfApi()
api.upload_folder(
    folder_path="vishal_model",
    repo_id="vishalgupta/t5-base-trained-vishal",
)


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

'https://huggingface.co/vishalgupta/t5-base-trained-vishal/tree/main/'

In [ ]:
from transformers import AutoTokenizer

autotokenizer = AutoTokenizer.from_pretrained("/content/vishal_model")